This contains ONLY the code to create a 4D image of 19 PET tracer images upsampled to 1mm merged together in a pseudo-timeseries, masked to a MNI152 1mm brain mask. The images used are for the following receptors and transporters:

D1,
D2(),
DAT,
NET,
5HT1A,
5HT1B(),
5HT2A,
5HT4,
5HT6,
5HTT,
alpha4beta2,
M1,
VAChT(),
NMDA,
mGluR5(),
GABAA/BZ,
H3,
CB1,
MOR(Mu Opioid Receptor)

() = made of >1 PET scan

Pipeline is as follows:

1. Collate raw neuromaps images (done in /rds/general/project/c3nl_scott_students/live/sankeith/neuromaps_raw)
2. Resample to 1mm MNI space (pays to be paranoid) via FLIRT. First to 1mm, then to MNI152 space. Restrict DoF so that Galovic's brain mask is registered appropriately
3. Z-score maps and check z scoring works (mean = 0, stdev = 1)
4. for neurotransmitters/receptors with >1 PET image corresponding to them, fslmerge -t, then fsl -Tmean them to get a 3D image
5. fslmerge and then check mean voxel value and number of regressors involved in the final image

In [7]:
%%bash
module load Python/3.13.1-GCCcore-14.2.0
python3 -m venv ~/venv_1
source ~/venv_1/bin/activate
pip install --upgrade pip
pip install nibabel
pip install ipykernel
pip install scipy
pip install pandas
python -m ipykernel install --user --name=venv_1 --display-name "Python (venv_1)"



3.13/site-packages (from ipykernel) (0.1.7)nline>=0.1 in ./venv_1/lib/python
ykernel) (25.0)eady satisfied: packaging in ./venv_1/lib/python3.13/site-packages (from ip
 ipykernel) (7.0.0) satisfied: psutil in ./venv_1/lib/python3.13/site-packages (from
from ipykernel) (5.14.3)sfied: traitlets>=5.4.0 in ./venv_1/lib/python3.13/site-packages (
site-packages (from ipython>=7.23.1->ipykernel) (0.6.3)ib/python3.13/
 pandas) (2025.2)dy satisfied: tzdata>=2022.7 in ./venv_1/lib/python3.13/site-packages (from
Installed kernelspec venv_1 in /rds/general/user/sk4724/home/.local/share/jupyter/kernels/venv_1


1. Collate raw neuromaps images (done in /rds/general/project/c3nl_scott_students/live/sankeith/neuromaps_raw)
2. Resample to 1mm MNI space (pays to be paranoid) via FLIRT. First to 1mm (1mm_ prefix), then to MNI152 space(F_1mm_ prefix). Restrict DoF so that Galovic's brain mask is registered appropriately
3. Z-score maps and check z scoring works (mean = 0, stdev = 1) (FZ_1mm_ prefix)

In [1]:
%%bash
module load fsl > /dev/null 2>&1
module load FSL/6.0.5.1-foss-2021a > /dev/null 2>&1
indir=/rds/general/project/c3nl_scott_students/live/sankeith/neuromaps_raw
outdir=/rds/general/project/c3nl_scott_students/live/sankeith/my_1mm_MNI152_maps
mask=/rds/general/project/c3nl_scott_students/live/sankeith/masks/MNI152_T1_1mm_brain_mask.nii.gz
ref=/rds/general/project/c3nl_scott_students/live/sankeith/standards/MNI152_T1_1mm_brain.nii.gz
mkdir -p "$outdir"
ls "$indir"|wc -l
for file in "$indir"/*;do
    inpath=$(basename "$file" .nii.gz);
    flirt -in "$file" -ref "$ref" -applyisoxfm 1.0 -nosearch -out ${outdir}/1mm_${inpath}.nii.gz
    flirt -in ${outdir}/1mm_${inpath}.nii.gz -ref "$ref" -dof 6 -out ${outdir}/F_1mm_${inpath}.nii.gz
    echo "flirt done for ${inpath}"
    mean=$(fslstats "${outdir}/F_1mm_${inpath}.nii.gz" -k "$mask" -m)
    std=$(fslstats "${outdir}/F_1mm_${inpath}.nii.gz" -k "$mask" -s )
    echo "Mean: ${mean}"
    echo "Standard Deviation: ${std}"
    fslmaths "${outdir}/F_1mm_${inpath}.nii.gz" -sub ${mean} -div ${std} -mas "$mask" "${outdir}/FZ_1mm_${inpath}.nii.gz"
    echo "z-scoring complete for ${inpath} - now validating"
    zscore_mean=$(fslstats "${outdir}/FZ_1mm_${inpath}.nii.gz" -k "$mask" -m )
    zscore_stdev=$(fslstats "${outdir}/FZ_1mm_${inpath}.nii.gz" -k "$mask" -s )
    echo "Z-Scored Mean: ${zscore_mean}"
    echo "Z-Scored Standard Deviation: ${zscore_stdev}"
done
rm -rf ${outdir}/.ipynb* ##there's usually a really annoying .ipynb_checkpoints folder that I delete because it complicates things it doesn't seem do anything
ls -l "$outdir"|wc -l

33

Final result: 
0.999986 0.005209 -0.000966 -6.995241 
-0.005224 0.999855 -0.016228 -3.544079 
881 0.016233 0.999868 -7.339391 
0.000000 0.000000 0.000000 1.000000 

flirt done for source-aghourian2017_desc-feobv_space-MNI152_res-1mm_feature
Mean: 25.052062 
Standard Deviation: 13.891708 
z-scoring complete for source-aghourian2017_desc-feobv_space-MNI152_res-1mm_feature - now validating
Z-Scored Mean: 0.000000 
Z-Scored Standard Deviation: 1.000000 

Final result: 
0.999981 0.004909 -0.003786 -0.281421 
-0.004940 0.999954 -0.008248 1.050702 
46 0.008266 0.999959 -0.629834 
0.000000 0.000000 0.000000 1.000000 

flirt done for source-alarkurtti2015_desc-raclopride_space-MNI152_res-3mm_feature
Mean: 18.367347 
Standard Deviation: 7.666867 
z-scoring complete for source-alarkurtti2015_desc-raclopride_space-MNI152_res-3mm_feature - now validating
Z-Scored Mean: -0.000000 
Z-Scored Standard Deviation: 1.000000 

Final result: 
0.999972 0.006206 0.004093 -7.243556 
-0.006182 0.999964 -0.0

In [1]:
import os
import nibabel as nib
##there may be a way of doing this in bash, but this is what stack overflow showed me so I'm using that
counter = 0
tot_counter = 0
folder = '/rds/general/project/c3nl_scott_students/live/sankeith/my_1mm_MNI152_maps/'
dodgy_files=[] ##essentially any non niftis
for filename in os.listdir(folder):
    tot_counter+=1
    if filename.endswith('.nii') or filename.endswith('.nii.gz'):
        file = os.path.join(folder, filename)
        img = nib.load(file)
        voxel_sizes = img.header.get_zooms()[:3]
        if voxel_sizes != (1.0, 1.0, 1.0):
            prefix = '_'.join(filename.split('_')[0:2])
            print(f"Voxel size for {prefix} (x, y, z):", voxel_sizes)
            counter += 1
    else:
        print(f"SKIPPING non-NIfTI file: {filename}")
        dodgy_files.append(filename)

print(f"\nTotal maps not in 1mm resolution: {counter}")
print(f"\nTotal maps: {tot_counter}")
print(f'Dodgy files: {len(dodgy_files)}')



Total maps not in 1mm resolution: 0

Total maps: 99
Dodgy files: 0


2. Collate FZ_ file paths in a txt

In [2]:
%%bash
## i rerun this later, but this helps me get filepaths for merging PET scans of the same tracer looking at the same receptor/transporter
dir=/rds/general/project/c3nl_scott_students/live/sankeith/my_1mm_MNI152_maps/
outfile=/rds/general/project/c3nl_scott_students/live/sankeith/fz_all_neuromaps.txt

# Clear existing file if needed
> "$outfile"

# Find files matching FZ_*.nii.gz and write to text file
find "$dir" -type f -name "FZ_1mm_*.nii.gz" >> "$outfile"

# Count how many were found
wc -l<"$outfile"


33


3.Check z scoring was successful

In [3]:
%%bash
## I've had to add this as a debugging step from the time the HPC refused to co-operate because of disk quota issues
dir=/rds/general/project/c3nl_scott_students/live/sankeith/my_1mm_MNI152_maps

d2_maps=(
"${dir}/FZ_1mm_source-smith2017_desc-flb457_space-MNI152_res-1mm_feature.nii.gz"
"${dir}/FZ_1mm_source-sandiego2015_desc-flb457_space-MNI152_res-1mm_feature.nii.gz"
)

fiveht1b_maps=(
"${dir}/FZ_1mm_source-gallezot2010_desc-p943_space-MNI152_res-1mm_feature.nii.gz"
"${dir}/FZ_1mm_source-savli2012_desc-p943_space-MNI152_res-3mm_feature.nii.gz"
)

vacht_maps=(
"${dir}/FZ_1mm_source-aghourian2017_desc-feobv_space-MNI152_res-1mm_feature.nii.gz"
"${dir}/FZ_1mm_source-bedard2019_desc-feobv_space-MNI152_res-1mm_feature.nii.gz"
"${dir}/FZ_1mm_source-tuominen_desc-feobv_space-MNI152_res-2mm_feature.nii.gz"
)

mglur5_maps=(
"${dir}/FZ_1mm_source-smart2019_desc-abp688_space-MNI152_res-1mm_feature.nii.gz"
"${dir}/FZ_1mm_source-rosaneto_desc-abp688_space-MNI152_res-1mm_feature.nii.gz"
"${dir}/FZ_1mm_source-dubois2015_desc-abp688_space-MNI152_res-1mm_feature.nii.gz"
)

echo "Checking if all files exist..."
for f in "${d2_maps[@]}" "${fiveht1b_maps[@]}" "${vacht_maps[@]}" "${mglur5_maps[@]}"; do
    if [ ! -f "$f" ]; then
        echo "❌ MISSING: $f"
    else
        echo "✅ Found: $f"
    fi
done


Checking if all files exist...
✅ Found: /rds/general/project/c3nl_scott_students/live/sankeith/my_1mm_MNI152_maps/FZ_1mm_source-smith2017_desc-flb457_space-MNI152_res-1mm_feature.nii.gz
students/live/sankeith/my_1mm_MNI152_maps/FZ_1mm_source-sandiego2015_desc-flb457_space-MNI152_res-1mm_feature.nii.gz
s/FZ_1mm_source-gallezot2010_desc-p943_space-MNI152_res-1mm_feature.nii.gz152_map
roject/c3nl_scott_students/live/sankeith/my_1mm_MNI152_maps/FZ_1mm_source-savli2012_desc-p943_space-MNI152_res-3mm_feature.nii.gz
✅ Found: /rds/general/project/c3nl_scott_students/live/sankeith/my_1mm_MNI152_maps/FZ_1mm_source-aghourian2017_desc-feobv_space-MNI152_res-1mm_feature.nii.gz
tt_students/live/sankeith/my_1mm_MNI152_maps/FZ_1mm_source-bedard2019_desc-feobv_space-MNI152_res-1mm_feature.nii.gz
s/FZ_1mm_source-tuominen_desc-feobv_space-MNI152_res-2mm_feature.nii.gzMNI152_map
ect/c3nl_scott_students/live/sankeith/my_1mm_MNI152_maps/FZ_1mm_source-smart2019_desc-abp688_space-MNI152_res-1mm_feature.nii.gz

4. for neurotransmitters/receptors with >1 PET image corresponding to them, fslmerge -t, then fsl -Tmean them to get a 3D image

In [4]:
%%bash
module load fsl > /dev/null 2>&1
module load FSL/6.0.5.1-foss-2021a > /dev/null 2>&1

dir=/rds/general/project/c3nl_scott_students/live/sankeith/my_1mm_MNI152_maps

d2_maps=(
"${dir}/FZ_1mm_source-smith2017_desc-flb457_space-MNI152_res-1mm_feature.nii.gz"
"${dir}/FZ_1mm_source-sandiego2015_desc-flb457_space-MNI152_res-1mm_feature.nii.gz"
)

fiveht1b_maps=(
"${dir}/FZ_1mm_source-gallezot2010_desc-p943_space-MNI152_res-1mm_feature.nii.gz"
"${dir}/FZ_1mm_source-savli2012_desc-p943_space-MNI152_res-3mm_feature.nii.gz"
)

vacht_maps=(
"${dir}/FZ_1mm_source-aghourian2017_desc-feobv_space-MNI152_res-1mm_feature.nii.gz"
"${dir}/FZ_1mm_source-bedard2019_desc-feobv_space-MNI152_res-1mm_feature.nii.gz"
"${dir}/FZ_1mm_source-tuominen_desc-feobv_space-MNI152_res-2mm_feature.nii.gz"
)

mglur5_maps=(
"${dir}/FZ_1mm_source-smart2019_desc-abp688_space-MNI152_res-1mm_feature.nii.gz"
"${dir}/FZ_1mm_source-rosaneto_desc-abp688_space-MNI152_res-1mm_feature.nii.gz"
"${dir}/FZ_1mm_source-dubois2015_desc-abp688_space-MNI152_res-1mm_feature.nii.gz"
)

d2=${dir}/FZ_1mm_d2_maps.nii.gz
fiveht1b=${dir}/FZ_1mm_5ht1b_maps.nii.gz
vacht=${dir}/FZ_1mm_vacht_maps.nii.gz
mglur_5=${dir}/FZ_1mm_mglur5_maps.nii.gz

# Intermediate files
tmp_d2=${dir}/FZ_d2_tmp.nii.gz
tmp_fiveht1b=${dir}/FZ_5ht1b_tmp.nii.gz
tmp_vacht=${dir}/FZ_vacht_tmp.nii.gz
tmp_mglur5=${dir}/FZ_mglur5_tmp.nii.gz

# D2
fslmerge -t "${tmp_d2}" "${d2_maps[@]}"
fslmaths "${tmp_d2}" -Tmean "${d2}"
rm -f "${tmp_d2}"

# 5HT1B
fslmerge -t "${tmp_fiveht1b}" "${fiveht1b_maps[@]}"
fslmaths "${tmp_fiveht1b}" -Tmean "${fiveht1b}"
rm -f "${tmp_fiveht1b}"

# VAChT
fslmerge -t "${tmp_vacht}" "${vacht_maps[@]}"
fslmaths "${tmp_vacht}" -Tmean "${vacht}"
rm -f "${tmp_vacht}"

# mGluR5
fslmerge -t "${tmp_mglur5}" "${mglur5_maps[@]}"
fslmaths "${tmp_mglur5}" -Tmean "${mglur_5}"
rm -f "${tmp_mglur5}"

echo "done!"

done!


In [5]:
%%bash
## told you i would rerun it
dir=/rds/general/project/c3nl_scott_students/live/sankeith/my_1mm_MNI152_maps/
outfile=/rds/general/project/c3nl_scott_students/live/sankeith/fz_all_neuromaps.txt

# Clear existing file if needed
> "$outfile"

# Find files matching FZ_*.nii.gz and write to text file
find "$dir" -type f -name "FZ_*.nii.gz" >> "$outfile"

# Count how many were found
wc -l<"$outfile" #it increases by 4, as expected and desired

37


In [6]:
%%bash
mask=/rds/general/project/c3nl_scott_students/live/sankeith/masks/MNI152_T1_1mm_brain_mask.nii.gz
dir=/rds/general/project/c3nl_scott_students/live/sankeith/my_1mm_MNI152_maps
d2=${dir}/FZ_1mm_d2_maps.nii.gz
fiveht1b=${dir}/FZ_1mm_5ht1b_maps.nii.gz
vacht=${dir}/FZ_1mm_vacht_maps.nii.gz
mglur_5=${dir}/FZ_1mm_mglur5_maps.nii.gz

module load fsl > /dev/null 2>&1
module load FSL/6.0.5.1-foss-2021a > /dev/null 2>&1

fslstats ${d2} -k "$mask" -m
fslstats ${fiveht1b} -k "$mask" -m
fslstats ${vacht} -k "$mask" -m
fslstats ${mglur_5} -k "$mask" -m

-0.000000 
-0.000000 
0.000000 
0.000001 


5. fslmerge and then check mean voxel value and number of regressors involved in the final image

In [7]:
%%bash
dir=/rds/general/project/c3nl_scott_students/live/sankeith/my_1mm_MNI152_maps
maps=(${dir}/FZ_1mm_source-kaller2017_desc-sch23390_space-MNI152_res-3mm_feature.nii.gz 
     ${dir}/FZ_1mm_d2_maps.nii.gz 
${dir}/FZ_1mm_source-dukart2018_desc-fpcit_space-MNI152_res-3mm_feature.nii.gz 
${dir}/FZ_1mm_source-ding2010_desc-mrb_space-MNI152_res-1mm_feature.nii.gz 
${dir}/FZ_1mm_source-savli2012_desc-way100635_space-MNI152_res-3mm_feature.nii.gz 
${dir}/FZ_1mm_5ht1b_maps.nii.gz 
${dir}/FZ_1mm_source-beliveau2017_desc-cimbi36_space-MNI152_res-1mm_feature.nii.gz 
${dir}/FZ_1mm_source-beliveau2017_desc-sb207145_space-MNI152_res-1mm_feature.nii.gz 
${dir}/FZ_1mm_source-radnakrishnan2018_desc-gsk215083_space-MNI152_res-1mm_feature.nii.gz 
${dir}/FZ_1mm_source-beliveau2017_desc-dasb_space-MNI152_res-1mm_feature.nii.gz 
${dir}/FZ_1mm_source-hillmer2016_desc-flubatine_space-MNI152_res-1mm_feature.nii.gz 
${dir}/FZ_1mm_source-naganawa2020_desc-lsn3172176_space-MNI152_res-1mm_feature.nii.gz 
${dir}/FZ_1mm_vacht_maps.nii.gz 
${dir}/FZ_1mm_source-galovic2021_desc-ge179_space-MNI152_res-1mm_feature.nii.gz 
${dir}/FZ_1mm_mglur5_maps.nii.gz 
${dir}/FZ_1mm_source-norgaard2021_desc-flumazenil_space-MNI152_res-1mm_feature.nii.gz 
${dir}/FZ_1mm_source-gallezot2017_desc-gsk189254_space-MNI152_res-1mm_feature.nii.gz 
${dir}/FZ_1mm_source-normandin2015_desc-omar_space-MNI152_res-1mm_feature.nii.gz 
${dir}/FZ_1mm_source-kantonen2020_desc-carfentanil_space-MNI152_res-3mm_feature.nii.gz)

final_map=/rds/general/project/c3nl_scott_students/live/sankeith/2_FINAL_NEUROMAPS_REGRESSOR.nii.gz

module load fsl > /dev/null 2>&1
module load FSL/6.0.5.1-foss-2021a > /dev/null 2>&1

fslmerge -t "${final_map}" "${maps[@]}" 

In [8]:
%%bash
module load fsl > /dev/null 2>&1
module load FSL/6.0.5.1-foss-2021a > /dev/null 2>&1

zscores=$(fslstats -t "/rds/general/project/c3nl_scott_students/live/sankeith/2_FINAL_NEUROMAPS_REGRESSOR.nii.gz" -k /rds/general/project/c3nl_scott_students/live/sankeith/masks/MNI152_T1_1mm_brain_mask.nii.gz -m)
echo "$zscores"
echo "###"
echo "$zscores" | wc -w

0.000000 
-0.000000 
-0.000000 
-0.000000 
0.000000 
-0.000000 
-0.000000 
0.000000 
-0.000001 
00000 
0.000000 
-0.000000 
0.000000 
0.000000 
0.000001 
0.000000 
0.000000 
0.000002 
-0.000000 
##
19


In [63]:
%%bash
module load fsl > /dev/null 2>&1
module load FSL/6.0.5.1-foss-2021a > /dev/null 2>&1

input_img="/rds/general/project/c3nl_scott_students/live/sankeith/2_FINAL_NEUROMAPS_REGRESSOR.nii.gz"
mask="/rds/general/project/c3nl_scott_students/live/sankeith/masks/MNI152_T1_1mm_brain_mask.nii.gz"
output_img="/rds/general/user/sk4724/home/neuromaps_std_masked.nii.gz"

##Get an mean map of the 4D regressor image so i can see the Mean and the STD.

fslmaths "$input_img" -Tmean -mas "$mask" "$output_img"

# Optionally, print the mean value of the STD image inside the mask. I'm also gonna run fslmeants 
fslmeants -i "$input_img" -m "$mask"
fslstats "$output_img" -k "$mask" -m
fslstats "$output_img" -k "$mask" -s


0.000000 
-0.000000 
-0.000000 
-0.000000 
0.000000 
-0.000000 
-0.000000 
0.000000 
-0.000001 
00000 
0.000000 
-0.000000 
0.000000 
0.000000 
0.000001 
0.000000 
0.000000 
0.000002 
-0.000000 

0.000000 
0.656378 
